# Kaggle contest training

The URL of the contest: https://www.kaggle.com/c/nlp-getting-started/overview

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-d9648ade-c31e-ab81-be67-d6db48bddb26)


In [ ]:
## Get helper functions
!wget https://raw.githubusercontent.com/BaoLocPham/Tensorflow_Deep_Learning/main/Utils/helper_functions.py

--2021-08-17 11:56:26--  https://raw.githubusercontent.com/BaoLocPham/Tensorflow_Deep_Learning/main/Utils/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11240 (11K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.98K  --.-KB/s    in 0s      

2021-08-17 11:56:26 (31.2 MB/s) - ‘helper_functions.py’ saved [11240/11240]



In [ ]:
# import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get text dataset

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data
unzip_data("nlp_getting_started.zip")

--2021-08-17 11:56:29--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-08-17 11:56:29 (108 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import pandas as pd

test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
sample_submit_df = pd.read_csv("sample_submission.csv")

In [ ]:
## shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


### Split data into training and validation set

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
len(train_sentences), len(val_sentences)

(6851, 762)

### Preprocess data

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
VOCAB_SIZE = 10000
MAX_LENGTH = 255

text_vectorizer = TextVectorization(max_tokens=VOCAB_SIZE,
                                       output_mode="int",
                                        output_sequence_length=MAX_LENGTH)
text_vectorizer.adapt(train_sentences)

In [ ]:
import random 

# Choose a random sentence from the training data and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Origin text:\n {random_sentence}\
          \n\nVectorize version:")
text_vectorizer([random_sentence])

Origin text:
 Now that's what you call a batting collapse #theashes          

Vectorize version:


<tf.Tensor: shape=(1, 255), dtype=int64, numpy=
array([[  48,  215,   55,   12,  425,    3, 6124,  155, 7673,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [ ]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common words
bottom_5_words = words_in_vocab[-5:] # least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=VOCAB_SIZE,
                             input_length=MAX_LENGTH,
                             output_dim=128,
                             embeddings_initializer="uniform")

In [ ]:
# Get a random sentence in the training set 
random_sentence = random.choice(train_sentences)
print(f"Original text: \n{random_sentence}\
\n\nEmbedded version:")
# Embed the ramdon sentence (turn it into dense vector of fixed size)
sample_vectorized = text_vectorizer([random_sentence])
print(f"Sample vectorized shape: {sample_vectorized.shape}")
sample_embedded = embedding(sample_vectorized)
sample_embedded

Original text: 
A poignant reminder that in war there are many casualties. http://t.co/Mwmt3BdR5L

Embedded version:
Sample vectorized shape: (1, 255)


<tf.Tensor: shape=(1, 255, 128), dtype=float32, numpy=
array([[[ 0.01427845,  0.00697158,  0.02343445, ..., -0.00322936,
          0.01475923, -0.04973565],
        [-0.0059801 , -0.02082992,  0.02860582, ...,  0.04510782,
          0.04722795, -0.03120303],
        [ 0.00378575, -0.04992047,  0.03134001, ...,  0.01537198,
         -0.01993409, -0.01678674],
        ...,
        [ 0.01989795,  0.02853802,  0.01318219, ...,  0.02367428,
         -0.02803319, -0.02223553],
        [ 0.01989795,  0.02853802,  0.01318219, ...,  0.02367428,
         -0.02803319, -0.02223553],
        [ 0.01989795,  0.02853802,  0.01318219, ...,  0.02367428,
         -0.02803319, -0.02223553]]], dtype=float32)>

## Optimized loading data

## Training model

Approachs:
* model 1: Feed forward model (Dense model) -> Tokenization + Embedding
* model 2: Conv1D -> Tokenization + Embedding



### Model 1 Feed forward model (Dense model) -> Tokenization + Embedding

In [ ]:
# num_class = len(train_sentences.class_names)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
# Create the model
inputs = layers.Input(shape=(1,),dtype=tf.string, name="input_layer")

x = text_vectorizer(inputs)
x = embedding(x)

x = layers.Flatten(name="flatten_layer")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1")

In [ ]:
model_1.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 255)               0         
_________________________________________________________________
embedding (Embedding)        (None, 255, 128)          1280000   
_________________________________________________________________
flatten_layer (Flatten)      (None, 32640)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 32641     
Total params: 1,312,641
Trainable params: 1,312,641
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5)

Epoch 1/5
215/215 [==============================] - 7s 18ms/step - loss: 0.5912 - accuracy: 0.6808 - val_loss: 0.4814 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3468 - accuracy: 0.8599 - val_loss: 0.4551 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2022 - accuracy: 0.9324 - val_loss: 0.5013 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1206 - accuracy: 0.9658 - val_loss: 0.5424 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0846 - accuracy: 0.9731 - val_loss: 0.5737 - val_accuracy: 0.7743


### Model 2: Conv1D -> Tokenization + Embedding

In [ ]:
# Create the model
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")

x = text_vectorizer(inputs)
x = embedding(x)

x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  padding="valid",
                  activation="relu")(x)

x = layers.GlobalAveragePooling1D(name="global_average_pool_1D")(x)
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2")

In [ ]:
# Compile the model
model_2.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 255)               0         
_________________________________________________________________
embedding (Embedding)        (None, 255, 128)          1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 251, 64)           41024     
_________________________________________________________________
global_average_pool_1D (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65  

In [ ]:
# fit the model
model_2_history = model_2.fit(train_sentences, 
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5)

Epoch 1/5
215/215 [==============================] - 32s 24ms/step - loss: 0.4317 - accuracy: 0.7920 - val_loss: 0.4956 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2278 - accuracy: 0.9132 - val_loss: 0.5514 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1583 - accuracy: 0.9396 - val_loss: 0.6647 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1141 - accuracy: 0.9585 - val_loss: 0.7941 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0924 - accuracy: 0.9658 - val_loss: 0.9116 - val_accuracy: 0.7664


### Model 3: Transfer learning: Feature extraction

Using Universal Sentence Embedding: https://tfhub.dev/google/universal-sentence-encoder/4

In [ ]:
import tensorflow_hub as hub

# Create a Keras layer using USE pretrained layer from tensorflow hub
feature_extractor_layer = hub.KerasLayer(handle="https://tfhub.dev/google/universal-sentence-encoder/4",
                                 input_shape=[],
                                dtype=tf.string,
                                trainable=False,
                                name="USE_layer")

In [ ]:
# Create a model

# inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")

# # stack the embedding layer
# x = embedding_layer(inputs)
# x = layers.Dense(64, activation="relu")(x)

# outputs = layers.Dense(1, activation="sigmoid")(x)

# model_3 = tf.keras.Model(inputs, outputs, name="model_3_feature_extract")

In [ ]:
# Create a model
model_3 = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
])

# Compile the model
model_3.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
model_3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_5 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5)

Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.5000 - accuracy: 0.7851 - val_loss: 0.4460 - val_accuracy: 0.8031
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4140 - accuracy: 0.8170 - val_loss: 0.4366 - val_accuracy: 0.8018
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.4008 - accuracy: 0.8216 - val_loss: 0.4367 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3928 - accuracy: 0.8257 - val_loss: 0.4286 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3853 - accuracy: 0.8298 - val_loss: 0.4308 - val_accuracy: 0.8084


In [ ]:
model_4 = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(256, activation="relu"),
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_4_feature_extraction")

# Compile the model
model_4.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=10)

Epoch 1/10
215/215 [==============================] - 6s 20ms/step - loss: 0.4627 - accuracy: 0.7986 - val_loss: 0.4267 - val_accuracy: 0.8084
Epoch 2/10
215/215 [==============================] - 4s 17ms/step - loss: 0.3878 - accuracy: 0.8308 - val_loss: 0.4172 - val_accuracy: 0.8084
Epoch 3/10
215/215 [==============================] - 4s 17ms/step - loss: 0.3521 - accuracy: 0.8521 - val_loss: 0.4290 - val_accuracy: 0.8163
Epoch 4/10
215/215 [==============================] - 4s 16ms/step - loss: 0.3029 - accuracy: 0.8765 - val_loss: 0.4240 - val_accuracy: 0.8268
Epoch 5/10
215/215 [==============================] - 3s 16ms/step - loss: 0.2411 - accuracy: 0.9067 - val_loss: 0.4761 - val_accuracy: 0.8110
Epoch 6/10
215/215 [==============================] - 3s 16ms/step - loss: 0.1782 - accuracy: 0.9383 - val_loss: 0.5402 - val_accuracy: 0.8084
Epoch 7/10
215/215 [==============================] - 4s 16ms/step - loss: 0.1286 - accuracy: 0.9578 - val_loss: 0.6334 - val_accuracy: 0.8005

### Model 5: Transfer learning: Fine tuning

In [ ]:
import tensorflow_hub as hub

# Create a Keras layer using USE pretrained layer from tensorflow hub
feature_extractor_fine_tune_layer = hub.KerasLayer(handle="https://tfhub.dev/google/universal-sentence-encoder/4",
                                 input_shape=[],
                                dtype=tf.string,
                                trainable=True,
                                name="USE_fine_tune_layer")

In [ ]:
model_5 = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(256, activation="relu"),
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_5_fine_tuning")

# Compile the model
model_5.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"]
)

In [ ]:
checkpoint_path = "model_5_fine_tune_new/checkpoint.ckpt"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         monitor="val_accuracy",
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         save_freq='epoch')

earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                          patience=10,
                                                          restore_best_weights=True)

In [ ]:
model_5_history_new = model_5.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=50,
                              callbacks=[checkpoint_callback, earlystopping_callback])

Epoch 1/50
215/215 [==============================] - 5s 18ms/step - loss: 0.6306 - accuracy: 0.7475 - val_loss: 0.5419 - val_accuracy: 0.7835
Epoch 2/50
215/215 [==============================] - 3s 15ms/step - loss: 0.4700 - accuracy: 0.8041 - val_loss: 0.4524 - val_accuracy: 0.7966
Epoch 3/50
215/215 [==============================] - 3s 16ms/step - loss: 0.4186 - accuracy: 0.8171 - val_loss: 0.4406 - val_accuracy: 0.8071
Epoch 4/50
215/215 [==============================] - 4s 16ms/step - loss: 0.4027 - accuracy: 0.8221 - val_loss: 0.4367 - val_accuracy: 0.8045
Epoch 5/50
215/215 [==============================] - 3s 16ms/step - loss: 0.3925 - accuracy: 0.8272 - val_loss: 0.4316 - val_accuracy: 0.8097
Epoch 6/50
215/215 [==============================] - 3s 16ms/step - loss: 0.3842 - accuracy: 0.8330 - val_loss: 0.4284 - val_accuracy: 0.8150
Epoch 7/50
215/215 [==============================] - 3s 15ms/step - loss: 0.3774 - accuracy: 0.8386 - val_loss: 0.4273 - val_accuracy: 0.8136

In [ ]:
model_5.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 14ms/step - loss: 0.4189 - accuracy: 0.8281


[0.4189437925815582, 0.8280839920043945]

In [ ]:
model_5.load_weights("/content/model_5_fine_tune/checkpoint.ckpt")

model_5.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 14ms/step - loss: 0.4264 - accuracy: 0.8333


[0.42635491490364075, 0.8333333134651184]

In [ ]:
model_5.save("/content/drive/MyDrive/Tensorflow Certificate Exam/Extra/NPL_disaster_tweets_kaggle_fine_tune")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Tensorflow Certificate Exam/Extra/NPL_disaster_tweets_kaggle_fine_tune/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Tensorflow Certificate Exam/Extra/NPL_disaster_tweets_kaggle_fine_tune/assets


### Model 6: LSTM model

In [ ]:
# Create models
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")

x = text_vectorizer(inputs)
x = embedding(x)

# x = layers.LSTM(units=64, return_sequences=True)(x)
# when you stack RNN cells together, you need to return sequence = True
x = layers.LSTM(64)(x)

# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)

model_6 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [ ]:
# compile the model
# Compile the model
model_6.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
checkpoint_path = "model_6_checkpoint/checkpoint.ckpt"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         monitor="val_accuracy",
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         save_freq='epoch')

earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                          patience=5,
                                                          restore_best_weights=True)

In [ ]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=20,
                              callbacks=[checkpoint_callback,
                                         earlystopping_callback])

Epoch 1/20
215/215 [==============================] - 17s 71ms/step - loss: 0.6832 - accuracy: 0.5725 - val_loss: 0.6932 - val_accuracy: 0.5433
Epoch 2/20
215/215 [==============================] - 15s 69ms/step - loss: 0.6829 - accuracy: 0.5733 - val_loss: 0.6910 - val_accuracy: 0.5433
Epoch 3/20
215/215 [==============================] - 15s 69ms/step - loss: 0.6830 - accuracy: 0.5733 - val_loss: 0.6899 - val_accuracy: 0.5433
Epoch 4/20
 86/215 [===========>..................] - ETA: 8s - loss: 0.6845 - accuracy: 0.5680

KeyboardInterrupt: ignored

### Create predictions

In [ ]:
import pandas as pd

test_df = pd.read_csv("test.csv")

test_questions =  test_df[["id", "text"]]

test_questions.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
text_question = test_df[["text"]].to_numpy()

In [ ]:
text_question.shape

(3263, 1)

In [ ]:
pred_prob = model_5.predict(text_question)

In [ ]:
pred = tf.squeeze(tf.cast(tf.round(pred_prob), tf.int16))
# pred[:10]
pred.shape

TensorShape([3263])

In [ ]:
id_questions = test_df[["id"]].to_numpy()
id_questions = tf.squeeze(id_questions)
id_questions.shape

TensorShape([3263])

In [ ]:
id_questions[:10]

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 0,  2,  3,  9, 11, 12, 21, 22, 27, 29])>

In [ ]:
demo_submission = pd.DataFrame({"id":id_questions, "target":pred})
demo_submission.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,0
6,21,0
7,22,0
8,27,0
9,29,0


In [ ]:
demo_submission.to_csv("demo_submission.csv",index=False)

In [ ]:
submission_1 = pd.DataFrame({"id":id_questions, "target":pred})

In [ ]:
submission_1.head(20)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [ ]:
submission_1.to_csv("submission_1.csv", index=False)

KeyboardInterrupt: ignored